# ДЗ 3. API VK

## VK API введение

Методы API, которые могут использоваться при работе с VK доступны по ссылке [API methods | Developers (vk.com)](https://vk.com/dev/methods). Этой ссылкой мы будем пользоваться постоянно. Каким образом будет продемонстировано далее.

Для простоты работы с VK API предлагаем воспользоваться библиотекой vk_api, которая представляет собой обертку для Python над функциями VK API и готова к использованию. Документация к библиотеке доступна тут - [Документация vk_api — vk_api 11.9.4 documentation (vk-api.readthedocs.io)](https://vk-api.readthedocs.io/en/latest/).

Установка vk_api является стандартной и выполняется при помощи pip install

In [ ]:
!pip install vk_api

## Получение доступа к выполнению запросов к VK API

Для получения информации о страницах, группах и т.д. VK требует авторизацию (точно так же как и с простым просмотром через браузер), поэтому необходимо показать VK, что у вас есть такое право. Достигается это при помощи точно такой же авторизации, какую вы выполняете через браузер или телефон, путем ввода логина и пароля.

Для начала импортируем библиотеку vk_api

In [ ]:
import vk_api
import dotenv
import os

dotenv.load_dotenv()

Теперь создадим объект сессии VK, указав для нее наш логин и пароль и попросим VK авторизовать нас.

In [ ]:
login = os.environ.get('LOGIN')
password = os.environ.get('PASSWORD')

vk_session = vk_api.VkApi(login, password)
vk_session.auth()

Теперь, выполнив авторизацию и получив право выполнять методы API, получим объект, который позволит нам работать с API.

In [ ]:
vk = vk_session.get_api()

## Как этим пользоваться

Теперь посмотрим какие методы нам доступны. Эта информация прописана на странице VK для разработчиков -  [API methods | Developers (vk.com)](https://vk.com/dev/methods).

Попробуем отыскать в этом обилии метод, который позволит нам получить пост по его идентификатору.

Для начала, что такое идентификатор поста?

Идентификатор поста это те циферки, которые видно, когда вы открываете этот пост.

Например:

![Untitled](https://github.com/hse-python-jour/hse_intro_to_data_journalism/blob/main/images/hw3/1.png?raw=true)

Идентификатором будет все, что стоит после слова wall. В данном случае он равен -41670861_2507472 (кстати, отрицательное число (-41670861) представляет собой идентификатор для групп, в то время как положительные идентификаторы представляют собой индентификатор для стен пользователей).

Теперь, зная этот идентификатор нам нужно как-то получить этот пост и загрузить его в питон. Вооружившись кружкой чая, гуглом и терпением обнаруживаем, что VK предоставляет именно тот метод, который нам нужен [wall.getById | Developers (vk.com)](https://vk.com/dev/wall.getById). На странице с описанием этого метода мы можем посмотреть на параметры, которые принимает этот метод

![Untitled](https://github.com/hse-python-jour/hse_intro_to_data_journalism/blob/main/images/hw3/2.png?raw=true)

Из описания параметров видим, что для получения информации о посте (или даже постах) нам нужно передать идентификаторы постов через запятые в параметр **posts**.

Итого, мы знаем что есть некий параметр posts, а так же имеем ссылку на страницу с документацией. Как нам теперь ~~со всем этим взлететь~~ этим воспользоваться?

Обратите внимание на то, как в ссылке значится этот метод - wall.getById

![Untitled](https://github.com/hse-python-jour/hse_intro_to_data_journalism/blob/main/images/hw3/3.png?raw=true)

vk_api дает нам возможность вызывать этот метод напрямую из объекта API (в нашем случае vk, который мы создали ранее) и передать в него параметры, которые указаны на странице с документацией

In [ ]:
vk.wall.getById(posts='-41670861_2507472') # vk.com/dev/wall.getById

В ответ нам вернется список словарей (для каждого из идентификатора, который был передан в параметр posts), которые содержат информацию о посте. Если был указан лишь один идентификатор, то список будет содержать лишь один словарь. Далее можно приступать к работе с полученными данными.

## Задания

### Задание 1 (0.5)

Найдите нужный метод в документации и получите 300 постов из вашей любимой группы ВК. По итогу должен получиться список из словарей, которые содержат информацию о посте (именно о посте, а не о результате запроса, который возвращается из метода)

In [ ]:
def fetchPosts(session_vk, id=-104168133, iterations=3, max=2):
    got = 0
    posts = []
    for i in range(0, iterations):
        ret = session_vk.wall.get(
            owner_id=id, offset=i*max, count=max, filter="owner")
        got = ret["count"]
        posts += list(ret.values())[1]
    return (got, posts)
    
(got, posts_list) = fetchPosts(vk, max=100)
print(f"Totally got: {got} posts")
print(posts_list[0])

### Задание 2 (2.5)

Допишите функцию, которая преобразует словарь для одного поста в словарь с ключами для текста поста, ссылкой на изображение в самом высоком разрешении (если есть, иначе пустая строка), количество комментариев, количество лайков, количество репостов, дату поста.

Если изображений в посте несколько, все равно возьмите то, у которого самое высокое разрешение.

Проверьте, что функция корректно отрабатывает на одном любом посте из списка, который вы получили в предыдущем задании

In [ ]:
from datetime import datetime
from typing import Dict, Any


def transform(post: Dict[str, Any]) -> Dict[str, Any]:
    pictures = list(
        map(
            lambda x: x["photo"],
            filter(
                lambda x: x and x["type"] == "photo",
                post.get("attachments") or [],
            ),
        )
    )
    max = {}
    maxRes = 0
    for pic in pictures:
        for size in pic.get('sizes') or []:
            currRes = size.get('width') * size.get('height')
            if (currRes > maxRes):
                max = size
                maxRes = currRes
    return {
        'pictureUrl': max.get('url') or '',
        'text': post.get('text'),
        'commentsCount': post.get('comments').get('count'),
        'likesCount': post.get('likes').get('count'),
        'repostsCount': post.get('reposts').get('count'),
        'date': datetime.fromtimestamp(post.get('date'))
    }

print(transform(posts_list[0]))

### Задание 3 (0.5)

При помощи функции map и функции, которую вы написали на предыдущем этапе, обработайте весь список из задания 1 и создайте новый обработанный список

In [ ]:
preprocessed_posts = list(map(transform, posts_list))

### Задание 4 (0.5)

При помощи Pandas создайте и сохраните в формат csv датасет последних 300 постов из вашей любимой группы ВК со следующими колонками: текст поста, ссылка на изображение в самом высоком разрешении (если есть, иначе пустая строка), количество комментариев, количество лайков, количество репостов, дату поста

In [ ]:
import pandas as pd

df = pd.DataFrame(data=preprocessed_posts)
df.to_csv("data.csv")

### Задание 5 (0.5)

Преобразуйте датасет из задания 4 так, чтобы в нем появилась новая колонка с указанием времени суток в которое было опубликован пост и новая колонка с указанием дня недели (см. datetime), постарайтесь избегать прямого использования циклов Python, попытайтесь решить задание лишь с помощью методов, которые предлагаются Pandas.

In [ ]:
df['time'] = pd.to_datetime(df['date'], format='%H:%M').dt.time
df['day_week']= df['date'].dt.day_name()
df.to_csv("data.csv")

### Задание 6 (0.5)

При помощи меотда hist ([pandas.DataFrame.hist — pandas 1.3.4 documentation (pydata.org)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html)) постройте гистограммы с распределением постинга постов по времени суток и с распределением постов по дням недели

In [ ]:
df = pd.DataFrame({
    'time': pd.to_datetime(df['date']).dt.hour,
    'day_week': pd.to_datetime(df['date']).dt.dayofweek,
})
df.hist()

### Задание 7 (2.5)

Из датасета получите список всех описаний постов, при помощи команд replace и split преобразуйте все посты в список из слов.

In [ ]:
import string
from typing import List


punctuation_tokens = string.punctuation  # получаем все знаки пунктуации


def get_wordlist(texts: List[str]) -> List[str]:
    all_words = []
    for text in texts:
        if not isinstance(text, str):
            continue
        new_text = text
        for ch in punctuation_tokens:
            new_text = new_text.replace(ch, " ")
        all_words += new_text.split()
    return all_words

In [ ]:
df_read = pd.read_csv("data.csv")
posts_csv_list = df_read['text'].tolist()
all_words = get_wordlist(posts_csv_list)
# print(all_words)

### Задание 8 (0.5)

При помощи списка из задания 7 постройте частотный словарь для вашей любимой группы, и выведите топ-20 слов с указанием их частотности

In [ ]:
from collections import Counter

words_dic = Counter(all_words).most_common(20)
print(words_dic)

### Задание 9 (2)

При помощи VK API получите комментарии (не менее 100) к любому посту и повторите задания 7 и 8.

In [ ]:
def fetchComments(session_vk, max=100, comment_id=None):
    print(f"comment_id = {comment_id}")
    ret = session_vk.wall.getComments(
        owner_id=-104168133, post_id=644134, count=max, comment_id=comment_id)
    got = ret["count"]
    comments = list(ret.values())[1]
    for comment in comments:
        thread = comment.get('thread') or {}
        new_id = comment.get('id')
        if new_id == None:
            continue
        if (thread.get('count') or 0) > 0:
            (got_more, comments_more) = fetchComments(
                session_vk, max, new_id)
            got += got_more
            comments += comments_more
    return (got, comments)

In [ ]:
(got, comments_list) = fetchComments(vk, 100)
preprocessed_comments = list(map(lambda x: {'text': x.get('text') or ''}, comments_list))
filtered_comments = list(filter(lambda x: x.get('text') != '', preprocessed_comments))
df = pd.DataFrame(data=filtered_comments)
df.to_csv("comments.csv")

df_read = pd.read_csv("comments.csv")
comments_csv_list = df_read['text'].tolist()
# print(comments_csv_list)
all_words = get_wordlist(comments_csv_list)
words_dic = Counter(all_words).most_common(20)
print(words_dic)